# Recuperación de recursos de la web
_Antonio Sarasa, Enrique Martín_

Se van a mostrar varias formas de recuperar recursos que se encuentran en la web:

* Recuperación con el módulo `urllib`
* Recuperación con el módulo `requests`

## Recuperación con el módulo `urllib`

urllib es una librería que permite tratar una página web de forma parecida a la apertura de un fichero, de forma que la librería gestiona de manera transparente todo lo referente al protocolo HTTP y los detalles de la cabecera.

Una vez que la página web ha sido abierta con urllib.request.urlopen, se puede tratar como un archivo y leer a través de ella usando un bucle for.

Cuando se recupera la página web, sólo se accede al contenido puesto las cabeceras aunque se envián,  el código de urllib se queda con ellas y sólo devuelve los datos.

En el siguiente ejemplo se recupera la información de una página web

In [1]:
from urllib.request import urlopen
resp = urlopen('https://learn.fdi.ucm.es')
print(type(resp))
# 'resp' es un objeto de tipo 'http.client.HTTPResponse' 
# https://docs.python.org/3/library/http.client.html#httpresponse-objects

print()
print(resp.headers)

body = resp.read()
print(body)
# 'read()' devuelve el contenido de la contestación como bytes

print()
print(body.decode('utf8'))
# Los bytes pueden ser decodificados a un str

<class 'http.client.HTTPResponse'>

Server: nginx/1.18.0 (Ubuntu)
Date: Fri, 04 Oct 2024 11:18:15 GMT
Content-Type: text/html; charset=utf-8
Content-Length: 4761
Connection: close
Expires: Fri, 04 Oct 2024 11:18:15 GMT
Cache-Control: max-age=0, no-cache, no-store, must-revalidate, private
Vary: Cookie, Accept-Language
X-Frame-Options: DENY
Content-Language: es
Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
X-Content-Type-Options: nosniff
Referrer-Policy: same-origin
Cross-Origin-Opener-Policy: same-origin
Set-Cookie: csrftoken=gngY94vO5k7mVwERV6SN8q5mK5FoQxiJ; expires=Fri, 03 Oct 2025 11:18:15 GMT; Max-Age=31449600; Path=/; SameSite=Lax; Secure
X-Frame-Options: DENY


b'\n\n\n<!DOCTYPE html>\n<html lang="es">\n<head>\n    <title>Learn SQL</title>\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n\n    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet"\n      

En el siguiente programa se procesa la información de un página web y se calcula la frecuencia de cada palabra:

In [2]:
from pprint import pp
from collections import Counter

manf = urlopen('http://learn.fdi.ucm.es').read().decode('utf8')
reps = Counter(manf.split())
pp(reps.most_common(10))

[('<div', 9),
 ('</div>', 9),
 ('<link', 5),
 ('<input', 5),
 ('rel="stylesheet"', 4),
 ('type="hidden"', 4),
 ('<button', 3),
 ('<span', 3),
 ('de', 3),
 ('tu', 3)]


La librería urllib también se puede utilizar para recuperar ficheros que no son de texto  como un archivo de imagen o de video, y para los que se requiere hacer una copia de la URL en un archivo local.

En estos casos se usa el método read para descargar el contenido completo del documento en una variable de tipo cadena y luego escribir la información a un archivo local.

## Recuperación con el módulo `requests`

El módulo requests permite bajarse archivos de la red de una forma transparente. No se encuentra instalado por defecto con Python.

La función get() del módulo requests toma una cadena que representa la url que se quiere descargar. El resultado de la llamada es un objeto de tipo Response que contiene la respuesta que el servidor web devuelve como respuesta a la petición.

In [3]:
import requests

res = requests.get("https://learn.fdi.ucm.es")
if res.status_code == requests.codes.ok:
    print(len(res.text))
    print(res.text[:250])

# ¡¡res.text ya está convenientemente decodificado!!

4757



<!DOCTYPE html>
<html lang="es">
<head>
    <title>Learn SQL</title>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">

    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstr


En el ejemplo además de recuperar la pagina descrita, se ha comprobado que la descarga se ha realizado con éxito chequeando el valor del atributo status_code del objeto Response. Si el valor que toma es requests.codes.ok significa que se ha realizado correctamente, y la descargada se ha almacenado como una cadena en la variable text del objeto Response.

Otra forma de comprobar si una descarga se ha realizado con éxito consiste en utilizar el método raise_for_status() del objeto Response. Este método lanzará una excepción si se ha producido algún error en la descarga y no hará nada en caso de que la descarga se haya realizado con éxito. Es por ello que una buena práctica consiste en encerrar la llamada al método raise_for_status() entre un try/except con el objetivo de tratar los casos en que se produzca una descarga errónea.

In [4]:
res = requests.get("http://learn.fdi.ucm.es")
try:
    res.raise_for_status()
    print("Todo OK")
except:
    print ("Hubo un problema")

Todo OK


Observar que es la llamada al método __raise_for_status()__ hay que realizarlo siempre después de llamar al método __requests.get()__ dado que el objetivo es asegurarse que la descarga se realizó con éxito antes de que el programa continúe su ejecución.

Una vez que ha realizado la descarga, lo que interesa es guardar el contenido en un archivo. Para ello se puede utilizar la función __open()__ y el método __write()__. En este sentido lo primero que hay que hacer es abrir un fichero en modo __“wb”__(escribir un modo binario), y a continuación escribir al fichero usando un bucle for que utilice el método __iter_content__ del objeto __Response__.

In [5]:
res = requests.get("http://learn.fdi.ucm.es")
try:
    res.raise_for_status()
    archivo = open("Archivo.txt","wb")
    for bloque in res.iter_content(10000):
       archivo.write(bloque)
    archivo.close()
except:
    print ("Hubo un problema")

Observar que el método iter_content recupera bloques de contenido en cada iteración a través del bucle. Cada bloque es un conjunto de bytes de datos en un tamaño igual al especificado. Así  mismo observar que el método __write()__ retorna el número de bytes escritos al fichero.

# Web Scraping

El “web scraping” consiste en escribir un programa que finge ser un navegador web y recupera páginas, examinando luego los datos de esas páginas para encontrar ciertos patrones.

Por ejemplo los buscadores:

* Buscan en el código de una página web, extraen los enlaces a otras páginas y recuperan esas páginas, extrayendo los enlaces que haya en ellas y así sucesivamente. 
* Usan la frecuencia con que las páginas que encuentra enlazan hacia una página concreta para calcular la “importancia” de esa página, y la posición en la que debe aparecer dentro de sus resultados de búsqueda.

Se van a estudiar dos formas de analizar páginas web:

* Mediante expresiones regulares.
* Mediante la librería BeautifulSoup.



## Expresiones regulares

Un forma fácil de analizar HTML consiste en utilizar expresiones regulares para hacer búsquedas repetidas que extraigan subcadenas coincidentes con un modelo concreto.

Considerar por ejemplo una página web que contiene enlaces y se quieren detectar dicho enlaces. Para ello se podría construir una expresión regular  que busque y extraiga los valores de los enlaces:

                                  href=("https://.+?")
                  
La expresión regular busca cadenas que comiencen por `href="https://`, seguido de uno o más caracteres (`.+?`), seguidos por otra comilla doble (`"`). El signo de interrogación añadido a `.+?` indica que se busca la cadena coincidente más pequeña posible.  Por último se añaden paréntesis a la expresión regular para indicar qué parte de la cadena localizada se quiere extraer.

In [6]:
import requests
from pprint import pp

url = 'https://ine.es/'
r = requests.get(url)
print(r.text[5800:6000])
# Vemos que no está decodificando bien. P.ej. "INE - Instituto Nacional de EstadÃstica"
print()

# Problema: el servidor del INE no incida el encoding en la cabecera HTTP de la contestación
pp(dict(r.headers))
print()

# Se debe indicar el charset en el "Content-Type", si no se asigna ISO-8859-1
print(r.encoding)

# Afortunadamente, requests puede indagar el encoding aparente de una contestación
print(r.apparent_encoding)
# y podemos forzarlo, en este caso a UTF8
r.encoding = r.apparent_encoding
print(r.text[5800:6000])

w.ine.es/#WebSite",            
    "name": "INE - Instituto Nacional de EstadÃ­stica",
    "inLanguage": "es",     
    "headline": "Instituto Nacional de EstadÃ­stica",
    "description" : "El I

{'Date': 'Fri, 04 Oct 2024 11:18:15 GMT',
 'Accept-Ranges': 'bytes',
 'Cache-Control': 'max-age=0',
 'Expires': 'Fri, 04 Oct 2024 11:18:15 GMT',
 'X-UA-Compatible': 'IE=edge',
 'Keep-Alive': 'timeout=15, max=100',
 'Connection': 'Keep-Alive',
 'Content-Type': 'text/html',
 'Set-Cookie': 'TS01f17082=018b1b3cc27ce080621aed638a0922c3ca578f39d38dfeb06a060533e19e594efcd3776d73bfec905619d27af3d1046f98c6758d05; '
               'Path=/; Domain=.ine.es',
 'Vary': 'Accept-Encoding',
 'Content-Encoding': 'gzip',
 'Transfer-Encoding': 'chunked'}

ISO-8859-1
utf-8
/#WebSite",            
    "name": "INE - Instituto Nacional de Estadística",
    "inLanguage": "es",     
    "headline": "Instituto Nacional de Estadística",
    "description" : "El INE elabora


In [7]:
import re

# Extraemos los enlaces HTTPS con una expresión regular
html = r.text
enlaces = re.findall('href=("https://.+?")', html)
pp(enlaces)

['"https://sede.ine.gob.es/ss/Satellite?c=Page&amp;cid=1254734719723&amp;pagename=SedeElectronica%2FSELayout&amp;lang=es_ES"',
 '"https://www.es-datalab.es/"',
 '"https://censo2021.csic.es/"',
 '"https://www.esc2024.eu/"',
 '"https://portal.mineco.gob.es/es-es/ministerio/participacionpublica/audienciapublica/Paginas/RD-aprueba-Clasificacion-Nacional-Actividades-Economicas-CNAE-2025.aspx"',
 '"https://portal.mineco.gob.es/es-es/ministerio/participacionpublica/audienciapublica/Paginas/Proyecto-Real-Decreto-aprueba-plan-estadistico-nacional-2025-2028.aspx"',
 '"https://sede.ine.gob.es/proyectos_investigacion"',
 '"https://censo2021.csic.es/"',
 '"https://public.tableau.com/views/EPAFlujosSankeyES_16252266850430/Dashboardccaa?:showVizHome=no&:embed=true#5"',
 '"https://public.tableau.com/views/CAUSASDEMUERTEactualizacin2018/Dashboard1?:showVizHome=no&:embed=true"',
 '"https://public.tableau.com/views/CFMSankeyES_16339576758090/Dashboard1?:showVizHome=no&:embed=true#3"',
 '"https://public.t

El método findall de las expresiones regulares proporciona una lista de todas las cadenas que coinciden con la expresión regular, devolviendo sólo el texto del enlace situado dentro de las comillas dobles.

Las expresiones regulares funcionan bien cuando el HTML está bien formado y es predecible. Sin embargo si el HTML no está bien construido, se pueden perder parte de los enlaces correctos, o terminar obteniendo datos erróneos.

## BeautifulSoup

BeautifulSoup es una biblioteca para extraer información de un página web. Su documentación completa está en https://www.crummy.com/software/BeautifulSoup/bs4/doc/ y se instala simplemente con:

     pip install beautifulsoup4

En este ejemplo se va analizar una página web y se van a extraer todos sus enlaces. Para ello el programa solicita una dirección web, luego abre la página web usando urllib, se leen los datos y se los pasa al analizador BeautifulSoup, que recupera todas las etiquetas de anclas(a) e imprime en pantalla el atributo href de cada una de ellas.

In [8]:
import requests 
from bs4 import BeautifulSoup
from pprint import pp

url = 'https://ine.es/'
r = requests.get(url)
r.encoding = r.apparent_encoding  # Corregimos el encoding
html = r.text
soup = BeautifulSoup(html, 'html.parser')

etiquetas = soup.find_all("a")
print(type(etiquetas[0]), etiquetas[0])
print()
for etiqueta in etiquetas:
    print(f"<<{etiqueta.text.strip()}>> --> {etiqueta.get('href', None)}")

<class 'bs4.element.Tag'> <a href="/"><img alt="SIGLAS Instituto Nacional de Estadística" src="/menus/_b/img/LogoINE.svg"/>
</a>

<<>> --> /
<<English>> --> /en/index.htm
<<>> --> /indiceweb.htm
<<Censo Electoral>> --> /dyngs/CEL/index.htm?cid=41
<<Sede electrónica>> --> https://sede.ine.gob.es/ss/Satellite?c=Page&cid=1254734719723&pagename=SedeElectronica%2FSELayout&lang=es_ES
<<Compartir>> --> javascript:void(0)
<<X>> --> #shareTwitter
<<Facebook>> --> #shareFacebook
<<Linkedin>> --> #shareLinkedin
<<WhatsApp>> --> #shareWhatsapp
<<Correo Electrónico>> --> #shareMail
<<Copiar al portapapeles>> --> #shareClipboard
<<48.797.875
						
Habitantes>> --> /dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177095&menu=ultiDatos&idp=1254735572981
<<1,5
						
IPC>> --> /dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176802&menu=ultiDatos&idp=1254735976607
<<3,1
						
PIB>> --> /dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736164439&menu=ultiDatos&idp=12547355765

Observar que el método BeautifulSoup() también admite como entrada un archivo html que esté descargado en el disco duro local.

In [9]:
from bs4 import BeautifulSoup

with open("Facultad de Informática.html", 'r', encoding='utf8') as html:
    soup = BeautifulSoup(html, 'html.parser')
    etiquetas = soup.find_all("a")
    for etiqueta in etiquetas:
        print(f"<<{etiqueta.text.strip()}>> --> {etiqueta.get('href', None)}")

<<Navegar identificado>> --> /login_sso/
<<>> --> https://www.ucm.es/
<<Facultad de Informática>> --> https://informatica.ucm.es/
<<Facultad>> --> /facultad
<<Presentación>> --> /presentacion
<<Bienvenida a la FdI>> --> /bienvenida
<<Gobierno>> --> /gobierno
<<Organización>> --> /organizacion
<<Eventos>> --> /eventos
<<+>> --> /facultad
<<Titulaciones>> --> /estudiar
<<Grado>> --> /grado
<<Máster>> --> /master
<<Doctorado>> --> /doctorado
<<Títulos propios>> --> /titulos-propios
<<+>> --> /estudiar
<<Estudiantes>> --> /estudiantes
<<Programacion docente>> --> /programacion-docente
<<Prácticas Externas>> --> /practicas-de-formacion
<<Ofertas de Empleo>> --> /ofertas-de-empleo
<<Secretaría de Estudiantes>> --> /secretaria
<<Movilidad>> --> /programas-de-movilidad
<<+>> --> /estudiantes
<<Servicios>> --> /asistencia
<<Infraestructuras>> --> /infraestructuras
<<Biblioteca>> --> http://biblioteca.ucm.es/fdi
<<Campus Virtual>> --> https://cv.ucm.es/CampusVirtual/jsp/index.jsp
<<Sede Electrón

El método BeautifulSoup() genera un objeto de tipo BeautifulSoup que tiene un conjunto de métodos que se pueden utilizar para localizar partes específicas de un documento html.

En el siguiente ejemplo se van a extraer varías partes de una etiqueta.

In [10]:
import requests 
from bs4 import BeautifulSoup

url = 'https://ine.es/'
r = requests.get(url)
r.encoding = r.apparent_encoding  # Corregimos el encoding
html = r.text
soup = BeautifulSoup(html, 'html.parser')
# Recupera todos los enlaces
etiquetas = soup.find_all("a")

# Accedemos a las partes de los nodos de etiqueta <a>
for etiqueta in etiquetas[:10]:
    print("Etiqueta:  ", etiqueta.name)
    print("URL:       ", etiqueta.get("href",None))
    print("Contenidos:", str(etiqueta.contents))
    print("Atributos :", etiqueta.attrs)
    print()

Etiqueta:   a
URL:        /
Contenidos: [<img alt="SIGLAS Instituto Nacional de Estadística" src="/menus/_b/img/LogoINE.svg"/>, '\n']
Atributos : {'href': '/'}

Etiqueta:   a
URL:        /en/index.htm
Contenidos: ['English']
Atributos : {'href': '/en/index.htm', 'lang': 'en', 'title': 'English Page', 'role': 'button'}

Etiqueta:   a
URL:        /indiceweb.htm
Contenidos: ['\n', <i class="ii ii-bars"></i>, '\n']
Atributos : {'href': '/indiceweb.htm', 'id': 'sidebarCollapse', 'class': ['btn', 'btn-info', 'no-events'], 'aria-label': 'Mostrar/ocultar el menú principal de navegación', 'title': 'Menú de navegación'}

Etiqueta:   a
URL:        /dyngs/CEL/index.htm?cid=41
Contenidos: ['Censo Electoral']
Atributos : {'class': ['tit'], 'href': '/dyngs/CEL/index.htm?cid=41', 'role': 'button', 'aria-haspopup': 'true', 'aria-expanded': 'false', 'aria-label': 'Censo Electoral', 'target': '_blank'}

Etiqueta:   a
URL:        https://sede.ine.gob.es/ss/Satellite?c=Page&cid=1254734719723&pagename=SedeE

### Imprimir un documento html

Existen varios métodos para imprimir un documento:

* La función __str()__ muestra el documento como una cadena, pero no elimina nodos que solo tengan un espacio en blanco ni añade espacios en blanco entre los nodos.

* La función __prettify()__ añade nuevas líneas y espacios para mostrar la estructura del documento html, y elimina nodos que solo contengan espacios en blanco.

In [11]:
from bs4 import BeautifulSoup

doc = "<html><h1>Cabecera</h1><p>Text"
soup = BeautifulSoup(doc, 'html.parser')

print(str(soup))
print()

print(soup.prettify())

<html><h1>Cabecera</h1><p>Text</p></html>

<html>
 <h1>
  Cabecera
 </h1>
 <p>
  Text
 </p>
</html>



### Analizar un documento html

BeautifulSoup toma un documento html y lo procesa a una estructura de datos en forma de árbol. Si el documento está bien formado el árbol se parece al documento original pero si no lo está entonces usa heurísticas para conseguir una estructura razonable. 

En este sentido se usa el siguiente conocimiento:

* Las tablas y listas de etiquetas tienen un orden de anidamiento natural. Por ejemplo &lt;TD> está en el interior de &lt;TR>.
* Los contenidos de una etiqueta &lt;SCRIPT> no deben ser parseados.
* Una etiqueta &lt;META> puede especificar una codificación del documento.

El objeto Beautifulsoup representa un árbol de procesamiento que contiene dos tipos de objetos:

* Objetos de tipo `Tag` que se corresponden con etiquetas o elementos del documento HTML
* Objetos de tipo `NavigableString` que se corresponden con cadenas de texto. Existen subclases de NavigableString que corresponden a construcciones especiales XML tales como CData, Comment, Declaration, and ProcessingInstruction.

Los elementos representados por los objetos de tipo `Tag` pueden tener asociados atributos y se puede acceder a ellos como si fuera un diccionario. Sin embargo los elementos representados por los objetos `NavigableString` no tienen asociados atributos.

Considerar el siguiente ejemplo para las explicaciones siguientes:

In [12]:
prueba = '''
<html>
  <head>
    <title>
    Título de la página
    </title>
    </head>
  <body>
    <p id="primerparrafo" align="center">
      Esto es un parrafo
      <b>
        one
      </b>
      .
    </p>
    <p id="segundoparrafo" align="blah">
      Esto es un parrafo
      <b>
        two
      </b>
      .
    </p>
  </body>
</html>
'''

En el siguiente ejemplo se recuperan las etiquetas “p” y se accede a sus atributos como si fuera un diccionario.

In [13]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(prueba, 'html.parser')
parrafos = soup.find_all("p")
for p in parrafos:
    print(p['id'])

primerparrafo
segundoparrafo


Los valores `Tag` pueden ser pasados a la función str() para mostrar las etiquetas que representan. Además los valores tag tienen un atributo llamado `attrs` que muestra todos los atributos HTML que tiene el elemento en forma de un diccionario.

Los objetos `Tag y `NavigableString` disponen de un conjunto de atributos y métodos:

* __parent__: Permite acceder al objeto que representa la etiqueta padre del objeto que representa a  una etiqueta. Permite navegar por el árbol de procesamiento.


In [14]:
soup.head.parent.name

'html'

* __contents__: Representa una lista ordenada de los objetos `Tag` y `NavigableString` contenidos dentro de un elemento. Solo el objeto que representa al árbol y los objetos tags poseen este atributo. Los objetos `NavigableString` no tienen el atributo `contents`, pues sólo tienen cadenas.


In [15]:
etiquetasp = soup.p  # Primer párrafo
for elem in etiquetasp.contents:
    print(f"* {type(elem)} -- {elem}")

* <class 'bs4.element.NavigableString'> -- 
      Esto es un parrafo
      
* <class 'bs4.element.Tag'> -- <b>
        one
      </b>
* <class 'bs4.element.NavigableString'> -- 
      .
    


In [16]:
etiquetasp.contents[1].contents

['\n        one\n      ']

In [17]:
# etiquetasp.contents[0].contents  # AttributeError: 'NavigableString' object has no attribute 'contents'

* __string__: Si un tag solo tiene un nodo hijo y se trata de una cadena, entonces se puede acceder al mismo mediante `tag.string` o mediante `tag.contents[0]`. Cuando existen varios hijos, y se trata de acceder al atributo string, devuelve como resultado el valor None. Los objetos `NavigableString` no tienen este atributo.

In [18]:
soup.b.string

'\n        one\n      '

In [19]:
soup.b.contents[0]

'\n        one\n      '

In [20]:
soup.p.string is None

True

In [21]:
soup.head.string is None

True

* __next_sibling__ y __previous_sibling__: Permite recuperar el objeto anterior o posterior que se encuentra al mismo nivel del objeto considerado. En el ejemplo anterior la etiqueta `<body>` está al mismo nivel que la etiqueta `<head>` pero esta última aparece antes.


In [22]:
soup.head.next_sibling.next_sibling  # head.next_sibling es un nodo de texto con un salto de línea

<body>
<p align="center" id="primerparrafo">
      Esto es un parrafo
      <b>
        one
      </b>
      .
    </p>
<p align="blah" id="segundoparrafo">
      Esto es un parrafo
      <b>
        two
      </b>
      .
    </p>
</body>

In [23]:
soup.head.previous_sibling  # Nodo de texto con salto de línea

'\n'

In [24]:
soup.head.previous_sibling.previous_sibling is None

True

* __next__ y __previous__: Permiten navegar en el árbol de procesamiento sobre los objetos en el orden en que fueron procesados en vez del orden dado por el árbol. En el ejemplo el objeto next al objeto que representa a &lt;HEAD> es el objeto que representa a &lt;TITLE> y no el objeto que representa a &lt;BODY>.

In [25]:
soup.head.next.next

<title>
    Título de la página
    </title>

In [26]:
soup.head.next.next.name

'title'

In [27]:
soup.head.previous.previous.name

'html'

Observaciones:

* Se puede iterar sobre el atributo contents de un objeto tag y tratarlo como una lista, y de forma similar se puede obtener el número de hijos mediante len(tag) o len(tag.contents).

In [28]:
for pos, elem in enumerate(soup.body):
    print(f"{pos} --> {type(elem)}")
    print(f"<<{elem}>>")
    print()

0 --> <class 'bs4.element.NavigableString'>
<<
>>

1 --> <class 'bs4.element.Tag'>
<<<p align="center" id="primerparrafo">
      Esto es un parrafo
      <b>
        one
      </b>
      .
    </p>>>

2 --> <class 'bs4.element.NavigableString'>
<<
>>

3 --> <class 'bs4.element.Tag'>
<<<p align="blah" id="segundoparrafo">
      Esto es un parrafo
      <b>
        two
      </b>
      .
    </p>>>

4 --> <class 'bs4.element.NavigableString'>
<<
>>



In [29]:
len(soup.body)

5

In [30]:
len(soup.body.contents)

5

* Se puede usar los nombres de un objeto tag como si fueran atributos del árbol de procesamiento o de un objeto tag. Siempre que se usa de esta forma, devuelve el primer nodo hijo cuyo nombre sea el considerado o bien retorna None si no existen hijos con ese nombre. En el ejemplo anterior para acceder a la etiqueta `<title> se puede ir a partir de la etiqueta `<head>` o bien directamente.


In [31]:
soup.head.title

<title>
    Título de la página
    </title>

In [32]:
soup.title

<title>
    Título de la página
    </title>

In [33]:
soup.img  # Si no existe, devuelve None

* Ahora se van a considerar los métodos que permiten buscar en el árbol de procesamiento: find_all y find.
* Solo se encuentran disponibles para el objeto que representa el árbol de procesamiento y en los objetos de tipo `Tag` pero no en los objetos de tipo `NavigableString`

El método find_all busca todos los objetos tag y NavigableString  que coinciden con un criterio dado desde un punto dado. Sus principales argumentos son:

    find_all(name, attrs, recursive, text, limit)

donde:
               
* Un nombre que restringe el conjunto de búsqueda por el nombre de las etiquetas.
* Pares atributo-valor que restringen el conjunto de búsqueda por los valores que toman los atributos de las etiquetas.
* El argumento “text” permite buscar objetos NavigableString , y puede tomar como valores una cadena, una expresión regular, una lista o diccionario, True o None o bien una expresión booleana. Cuando se usa este argumento, las restricciones sobre nombre o atributo no se tienen en cuenta.
* El argumento “recursive” que puede tomar los valores True o False que indica si busca por debajo del árbol o bien por los hijos inmediatos del árbol. Por defecto es True.
* El argumento limit permite parar la búsqueda cuando se han conseguido un número de coincidencias dado. Por defecto no tienen límite.




Algunos ejemplos de restricción por nombre:

* Argumento con el nombre de una etiqueta.

In [34]:
soup.find_all('b')  # Elementos de etiqueta <b>

[<b>
         one
       </b>,
 <b>
         two
       </b>]

* Argumento con una expresión regular

In [35]:
import re

etiquetasconb = soup.find_all(re.compile('^b'))  # Elementos de etiqueta <b*>
for tag in etiquetasconb:
    print(tag.name)

body
b
b


* Argumento con una lista o diccionario.

In [36]:
soup.find_all(['title', 'p'])  # Elementos <title> o <p>

[<title>
     Título de la página
     </title>,
 <p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>,
 <p align="blah" id="segundoparrafo">
       Esto es un parrafo
       <b>
         two
       </b>
       .
     </p>]

* Argumento con una expresión que evalúa a cierto o falso.

In [37]:
soup.find_all(lambda tag: len(tag.attrs) == 2)  # Elementos con dos atributos

[<p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>,
 <p align="blah" id="segundoparrafo">
       Esto es un parrafo
       <b>
         two
       </b>
       .
     </p>]

In [38]:
soup.find_all(lambda tag: len(tag.name)==1 and not tag.attrs)  # Elementos cuyo nombre es una única letra y no tienen atributos

[<b>
         one
       </b>,
 <b>
         two
       </b>]

Algunos ejemplos de restricción por atributo:

* Argumento que impone una condición al valor que toma un atributo.

In [39]:
soup.find_all(align="center")  # Elementos con el atributo align="center"

[<p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>]

* Argumento que impone una condición en forma de expresión regular al valor que toma un atributo.

In [40]:
soup.find_all(id=re.compile("afo$"))  # Elementos con el id terminado en "parra"

[<p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>,
 <p align="blah" id="segundoparrafo">
       Esto es un parrafo
       <b>
         two
       </b>
       .
     </p>]

In [41]:
soup.find_all(id=re.compile("^prim"))  # Elementos con el id empezando en "prim"

[<p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>]

* Argumento que impone una condición en forma de lista al valor que toma un atributo.

In [42]:
soup.find_all(align=["center","blah"])  # Elementos cuyo atributo align toma valores "center" o "blah"

[<p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>,
 <p align="blah" id="segundoparrafo">
       Esto es un parrafo
       <b>
         two
       </b>
       .
     </p>]

* Argumento que impone una condición en forma de una expresión booleano al valor que toma un atributo.

In [43]:
soup.find_all(align=lambda value: value and len(value) <5)  # Elementos cuyo atributo align cumple la función

[<p align="blah" id="segundoparrafo">
       Esto es un parrafo
       <b>
         two
       </b>
       .
     </p>]

* Argumento que iguala a True o None el valor de un atributo. True encaja con etiquetas que toman cualquier valor para el atributo y None encaja con etiquetas que no tienen valor para ese atributo.

In [44]:
soup.find_all(align=True)  # Elementos con atributo align

[<p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>,
 <p align="blah" id="segundoparrafo">
       Esto es un parrafo
       <b>
         two
       </b>
       .
     </p>]

In [45]:
etiquetas=soup.find_all(align=None) # Elementos sin atributo align
[eti.name for eti in etiquetas]

['html', 'head', 'title', 'body', 'b', 'b']

Algunos ejemplos del argumento text:

In [46]:
# Elementos de tipo texto que contienen "Esto"
elems = soup.find_all(string=re.compile("Esto"))
[(type(elem), elem) for elem in elems]

[(bs4.element.NavigableString, '\n      Esto es un parrafo\n      '),
 (bs4.element.NavigableString, '\n      Esto es un parrafo\n      ')]

In [47]:
# Todos los elementos de tipo texto
elems = soup.find_all(string=True)
[(type(elem), elem) for elem in elems]

[(bs4.element.NavigableString, '\n'),
 (bs4.element.NavigableString, '\n'),
 (bs4.element.NavigableString, '\n'),
 (bs4.element.NavigableString, '\n    Título de la página\n    '),
 (bs4.element.NavigableString, '\n'),
 (bs4.element.NavigableString, '\n'),
 (bs4.element.NavigableString, '\n'),
 (bs4.element.NavigableString, '\n      Esto es un parrafo\n      '),
 (bs4.element.NavigableString, '\n        one\n      '),
 (bs4.element.NavigableString, '\n      .\n    '),
 (bs4.element.NavigableString, '\n'),
 (bs4.element.NavigableString, '\n      Esto es un parrafo\n      '),
 (bs4.element.NavigableString, '\n        two\n      '),
 (bs4.element.NavigableString, '\n      .\n    '),
 (bs4.element.NavigableString, '\n'),
 (bs4.element.NavigableString, '\n'),
 (bs4.element.NavigableString, '\n')]

In [48]:
# Elementos de tipo texto con más de 8 letras al quitar caracteres en blanco por los lados
soup.find_all(string=lambda x: len(x.strip()) > 8 )

['\n    Título de la página\n    ',
 '\n      Esto es un parrafo\n      ',
 '\n      Esto es un parrafo\n      ']

Algunos ejemplos del argumento recursive:

In [49]:
[tag.name for tag in soup.html.find_all()]

['head', 'title', 'body', 'p', 'b', 'p', 'b']

In [50]:
[tag.name for tag in soup.html.find_all(recursive=False)]

['head', 'body']

Algunos ejemplos del argumento limit:

In [51]:
soup.find_all('p', limit=1)

[<p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>]

In [52]:
soup.find_all('p', limit=100)

[<p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>,
 <p align="blah" id="segundoparrafo">
       Esto es un parrafo
       <b>
         two
       </b>
       .
     </p>]

El método find tiene una estructura similar a find_all:
          
    find(name, attrs, recursive, text)

Es similar a find_all con la única diferencia de que en vez de recuperar todas las coincidencias, recupera sólo la primera. Tiene el mismo comportamiento que si el parámetro limit de find_all tomara el valor 1 salvo:
* En caso de tener éxito, `find_all` devolverá una lista unitaria y `find` devolverá directamente el elmento encontrado
* En caso de no tener éxito, `find_all` devolverá una lista vacía y `find` devolverá `None`


In [53]:
soup.find_all('p', limit=1)

[<p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>]

In [54]:
soup.find('p')

<p align="center" id="primerparrafo">
      Esto es un parrafo
      <b>
        one
      </b>
      .
    </p>

In [55]:
soup.p

<p align="center" id="primerparrafo">
      Esto es un parrafo
      <b>
        one
      </b>
      .
    </p>

In [56]:
soup.find_all('img', limit=1)

[]

In [57]:
soup.find('img') is None

True

In [58]:
soup.img is None

True

Observaciones:

* Puede ocurrir que algún atributo de una etiqueta coincida con las palabras reservadas en BeautifulSoup. En estos casos no se puede hacer una referencia directa al atributo y es necesario usar un atributo que tienen las etiquetas denominado attrs que actúa como un diccionario que hace referencia a los atributos de una etiqueta.

In [59]:
soup.find_all(attrs={"id":"primerparrafo"})

[<p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>]

* Se pueden combinar búsqueda por nombre y por atributo:

In [60]:
soup.find_all('p', {"align":"center"})

[<p align="center" id="primerparrafo">
       Esto es un parrafo
       <b>
         one
       </b>
       .
     </p>]

* find_parents(name, attrs, limit) y find_parent(name, attrs):                                                       Devuelve los padres de la etiqueta considerada que coinciden con los criterios de búsqueda.

In [61]:
etiqueta = soup.find('b')
[eti.name for eti in etiqueta.find_parents()]

['p', 'body', 'html', '[document]']

In [62]:
etiqueta.find_parent('body').name

'body'

### Ejemplo completo con BeautifulSoup: descargar imágenes de más de 200x200 píxeles de una web

Se crea un árbol de procesamiento con BeautifulSoup y se recuperan todas las etiquetas `img` con de tamaño suficientemente grande:

In [63]:
import requests
from bs4 import BeautifulSoup

html = requests.get("http://trenesytiempos.blogspot.com.es/").text
soup = BeautifulSoup(html,"html.parser")
etiquetas = soup.find_all('img', height=lambda x: int(x) > 200, width=lambda x: int(x) > 200)
len(etiquetas)

50

Recuperadas las URL de las direcciones que se quieren almacenar en el disco local, vamos una a una y las obtenemos con `requests` para escribir en un fichero diferente.

In [64]:
import requests

# Descargar todas las fotos tarda un rato largo
# Solo descargamos las 10 primeras
for i, elem in enumerate(etiquetas[:10]):
    with open(f"foto{i}.jpg", "wb") as archivo:
        print(f"{i}) Descargando foto {elem['src']}")
        res = requests.get(elem['src'])
        archivo.write(res.content)

0) Descargando foto https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEhFgOtS255QY_-65I4ooM2Usj1Q_0vf-cQqkUdyTQfPHoIDoaXYnojCSpAJuI8YYnX0Z_jvdUSJpbeFDl3UtFxo8V9TRX0CKJDEGT9HkdVCck82FRWoCYMfDogO9mfoaWZuHM7ruLvwRhwm9DtqbruIG9IeNBavefdvuvqe91ptmJFNworUWb3Msu6WVxV8/w640-h460/Captura%20de%20pantalla%202024-07-21%20a%20las%2011.51.56.png
1) Descargando foto https://blogger.googleusercontent.com/img/a/AVvXsEhSTfLylnhFZcDSvbedRgH2fxkW8iEcdMmeZw3Ybl6nZ54dPwUg64yBHG-n4aJ01LW8N8XIbSdlDgQ5bevoJr0Vyc7u3iLtt_tcbkvZmK5TzqRtz_Kq4VFy3byeqFcbEyz_zfRZ6_a_qDnnF8upYP60SvsyMh2JHJu4p3q8s55YWY4MvgcFWJcho7hZ2mVg=w640-h426
2) Descargando foto https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEgQjPx-KutB4XF53mkHHpYwY_iNfUajBVZ0F3dsLyHrw_Jh52-PYfFOZ4vSE5_jjYn9Zn1Khb8pKK4e4nP8eX_KAfETVFI9hygdVEe735tD-RcnhR869gNTWXKozHge7a5egJTH9dnNFnWwlVfITqpZkpnTSY0rDFTfJdRLbH5FO_JTucq-mxHaROtfyeHt/w640-h462/Captura%20de%20pantalla%202024-07-22%20a%20las%2011.17.14.png
3) Descargando foto https://blogger.googleuse

### Ejemplo completo con BeautifulSoup: temas principales en la web del INE

In [65]:
from bs4 import BeautifulSoup
import requests

BASE_URL = 'https://www.ine.es/'

res = requests.get(BASE_URL)
res.encoding = res.apparent_encoding  # Corregimos el encoding
soup = BeautifulSoup(res.text)
sec = soup.css.select_one("section#temas")
temas_li = sec.find_all('li')

[(tema.p.text, f"{BASE_URL}{tema.a.get('href')}") for tema in temas_li]

[('Agricultura y medio ambiente',
  'https://www.ine.es//dyngs/INEbase/es/categoria.htm?c=Estadistica_P&cid=1254735570567'),
 ('Ciencia y tecnología',
  'https://www.ine.es//dyngs/INEbase/es/categoria.htm?c=Estadistica_P&cid=1254735976151'),
 ('Demografía y población',
  'https://www.ine.es//dyngs/INEbase/es/categoria.htm?c=Estadistica_P&cid=1254734710984'),
 ('Economía',
  'https://www.ine.es//dyngs/INEbase/es/categoria.htm?c=Estadistica_P&cid=1254735570541'),
 ('Industria, energía y construcción',
  'https://www.ine.es//dyngs/INEbase/es/categoria.htm?c=Estadistica_P&cid=1254735570688'),
 ('Mercado laboral',
  'https://www.ine.es//dyngs/INEbase/es/categoria.htm?c=Estadistica_P&cid=1254735976594'),
 ('Servicios',
  'https://www.ine.es//dyngs/INEbase/es/categoria.htm?c=Estadistica_P&cid=1254735570703'),
 ('Nivel y condiciones de vida (IPC)',
  'https://www.ine.es//dyngs/INEbase/es/categoria.htm?c=Estadistica_P&cid=1254735976604'),
 ('Sociedad',
  'https://www.ine.es//dyngs/INEbase/es/ca

## MechanicalSoup

MechanicalSoup es una biblioteca Python que nos permite cierta automatización con el navegador para hacer _web scraping_, por ejemplo rellenar formularios y analizar la respuesta obtenida. Se instala con:

    pip install MechanicalSoup

Un aspecto interesante es que te puede devolver las páginas directamente como sopas de BeautifulSoup, por lo que podemos usar todo lo que hemos visto anteriormente para analizar el HTML obtenido. 

Veamos un ejemplo de cómo acceder al IPC entre dos momentos temporales usando la página oficial del Instituto Nacional de Estadística (INE).

In [66]:
import mechanicalsoup

url = 'https://www.ine.es/varipc/index.do'
browser = mechanicalsoup.StatefulBrowser()
browser.open(url)

# Seleccionamos el formulario de nombre 'VarIPCFormBean'
browser.select_form('form[name=VarIPCFormBean]')
browser.form.print_summary()

<select class="caja azul" id="mesini" name="idmesini"><option value="1">Enero</option>
<option value="2">Febrero</option>
<option value="3">Marzo</option>
<option value="4">Abril</option>
<option value="5">Mayo</option>
<option value="6">Junio</option>
<option value="7">Julio</option>
<option selected="" value="8">Agosto</option>
<option value="9">Septiembre</option>
<option value="10">Octubre</option>
<option value="11">Noviembre</option>
<option value="12">Diciembre</option></select>
<select class="caja azul" id="anyoini" name="anyoini"><option value="1961">1961</option>
<option value="1962">1962</option>
<option value="1963">1963</option>
<option value="1964">1964</option>
<option value="1965">1965</option>
<option value="1966">1966</option>
<option value="1967">1967</option>
<option value="1968">1968</option>
<option value="1969">1969</option>
<option value="1970">1970</option>
<option value="1971">1971</option>
<option value="1972">1972</option>
<option value="1973">1973</option>


In [67]:
# Rellenamos el formulario para conocer el IPC entre enero de 2021 y diciembre de 2022
browser["idmesini"] = "Enero"
browser["anyoini"] = "2021"
browser["idmesfin"] = "Diciembre"
browser["anyofin"] = "2022"
browser["ntipo"] = "1" # Tipo "General Nacional (desde enero de 1961)"
browser.form.print_summary()

<select class="caja azul" id="mesini" name="idmesini"><option selected="selected" value="1">Enero</option>
<option value="2">Febrero</option>
<option value="3">Marzo</option>
<option value="4">Abril</option>
<option value="5">Mayo</option>
<option value="6">Junio</option>
<option value="7">Julio</option>
<option value="8">Agosto</option>
<option value="9">Septiembre</option>
<option value="10">Octubre</option>
<option value="11">Noviembre</option>
<option value="12">Diciembre</option></select>
<select class="caja azul" id="anyoini" name="anyoini"><option value="1961">1961</option>
<option value="1962">1962</option>
<option value="1963">1963</option>
<option value="1964">1964</option>
<option value="1965">1965</option>
<option value="1966">1966</option>
<option value="1967">1967</option>
<option value="1968">1968</option>
<option value="1969">1969</option>
<option value="1970">1970</option>
<option value="1971">1971</option>
<option value="1972">1972</option>
<option value="1973">1973</

In [68]:
from pprint import pp

response = browser.submit_selected()

print(response.status_code)
pp(dict(response.headers))
pp(response.text[:1000])
response

200
{'Date': 'Fri, 04 Oct 2024 11:18:22 GMT',
 'Cache-Control': 'no-store',
 'Pragma': 'no-cache',
 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT',
 'Content-Type': 'text/html;charset=UTF-8',
 'Keep-Alive': 'timeout=15, max=99',
 'Connection': 'Keep-Alive',
 'Set-Cookie': 'TS01c34874=018b1b3cc21de9bcdc3803e9708979faf316facfd00203ae683c552ef4e1da18013de33e21c4654d97cb8ea067d4c43ff5aab6ebfd; '
               'Path=/; Domain=.www.ine.es',
 'Vary': 'Accept-Encoding',
 'Content-Encoding': 'gzip',
 'Transfer-Encoding': 'chunked'}
('\r\n'
 '\r\n'
 '\r\n'
 '\r\n'
 '\r\n'
 '\r\n'
 '\r\n'
 '\r\n'
 '\r\n'
 '\r\n'
 '<!DOCTYPE html>\r\n'
 '\r\n'
 '<html lang="es">\r\n'
 '<head>\r\n'
 '<title>Cálculo de variaciones del Indice de Precios de Consumo</title>\r\n'
 '\r\n'
 '<meta http-equiv="X-UA-Compatible" content="IE=edge">\n'
 '<meta charset="UTF-8">\n'
 '<link rel="shortcut icon" href="/menus/img/favicon.ico" '
 'type="image/x-icon">\n'
 '<link href="/menus/img/favicon.png" rel="apple-touch-icon">\n'
 

<Response [200]>

In [69]:
soup = browser.page  # Devuelve directamente una sopa de BeautifulSoup
casilla = soup.select_one("td.celdadato")  # Con select_one buscamos usando un selector CSS
ipc = casilla.contents[2].text.strip()
ipc

'12,6'

In [70]:
# Ojo que devuelve el decimal con coma, si queremos convertilo a float para operar o 
# comparar con este valor deberíamos reemplazar ',' por '.'
float(ipc.replace(',', '.'))

12.6